In [3]:
%load_ext cuml.accel
%load_ext cudf.pandas
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
import numpy as np
from fast_ml.model_development import train_valid_test_split
from transformers import Trainer, TrainingArguments, AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch import nn
from torch.nn.functional import softmax
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import datasets
import evaluate

[2025-06-01 18:12:36.206] [CUML] [info] cuML: Installed accelerator for sklearn.
[2025-06-01 18:13:46.029] [CUML] [info] cuML: Installed accelerator for umap.
[2025-06-01 18:13:46.120] [CUML] [info] cuML: Installed accelerator for hdbscan.
[2025-06-01 18:13:46.120] [CUML] [info] cuML: Successfully initialized accelerator.


/usr/local/lib/python3.11/dist-packages/cudf/pandas/__init__.py:65: UserWarning: cudf.pandas detected an already configured memory resource, ignoring 'CUDF_PANDAS_RMM_MODE'=managed_pool
  warnings.warn(


In [5]:
df = pd.read_csv('df_comentario_limpo.csv')
df

,Unnamed: 0,Name,Likes,Review_Title,Date,Review_Type,Review_Body,Review_Classification,Language,Restaurant_name,comentario_limpo
0,0,Wagner B,0,ALMOÇO MARAVILHOSO,04/2025,family,Local excepcional. Todos os comentários feitos...,5,pt,Floresta das escadinhas,local excepcional todos comentários feitos rea...
1,1,Matheus A,0,Ótimo restaurante,04/2025,family,"Ótimo lugar, excelente comida, custo benefício...",5,pt,Floresta das escadinhas,ótimo lugar excelente comida custo benefício m...
2,2,alan s,0,Excepcional,04/2025,couples,"Restaurante muito bom, agradavel e a manuela n...",5,pt,Floresta das escadinhas,restaurante bom agradavel manuela recebeu bem ...
3,3,Felipe A,0,Uma ótima opção no centro,04/2025,couples,"Tivemos uma ótima experiência, o atendimento é...",5,pt,Floresta das escadinhas,ótima experiência atendimento fascinante marav...
4,4,Lízzie F,0,Boa comida e preço,04/2025,friends,Fomos muito bem recebidas pelo Augusto \nÓtima...,5,pt,Floresta das escadinhas,bem recebidas augusto ótima comida ótimo preço...
...,...,...,...,...,...,...,...,...,...,...,...
13250,13250,Jerzy1969,0,They have lost attention to food quality and C...,undefined,undefined,We have stoopped for small meal with kids- had...,2,eng,Nicola,we have stoopped small meal with kids had tabl...
13251,13251,Valerie G,0,Try another restaurant,08/2011,none,We stayed in a hotel along the Avenida Liberda...,2,eng,Nicola,we stayed in hotel along the avenida liberdad ...
13252,13252,J and d.,0,Great location for Dining in Central Lisbon,09/2011,none,The location on a main square in Lisbon makes ...,4,eng,Nicola,the location on main square in lisbon makes th...
13253,13253,fridayinla,0,Average dining with below average service.,undefined,undefined,Although the restaurant has a charmingly class...,3,eng,Nicola,although the restaurant has charmingly classic...


In [6]:
#df['Review_Classification'] = df['Review_Classification'].str.replace(r'\s+de\s+(\d+)\s+bolhas', '', regex=True)

In [7]:
df

,Unnamed: 0,Name,Likes,Review_Title,Date,Review_Type,Review_Body,Review_Classification,Language,Restaurant_name,comentario_limpo
0,0,Wagner B,0,ALMOÇO MARAVILHOSO,04/2025,family,Local excepcional. Todos os comentários feitos...,5,pt,Floresta das escadinhas,local excepcional todos comentários feitos rea...
1,1,Matheus A,0,Ótimo restaurante,04/2025,family,"Ótimo lugar, excelente comida, custo benefício...",5,pt,Floresta das escadinhas,ótimo lugar excelente comida custo benefício m...
2,2,alan s,0,Excepcional,04/2025,couples,"Restaurante muito bom, agradavel e a manuela n...",5,pt,Floresta das escadinhas,restaurante bom agradavel manuela recebeu bem ...
3,3,Felipe A,0,Uma ótima opção no centro,04/2025,couples,"Tivemos uma ótima experiência, o atendimento é...",5,pt,Floresta das escadinhas,ótima experiência atendimento fascinante marav...
4,4,Lízzie F,0,Boa comida e preço,04/2025,friends,Fomos muito bem recebidas pelo Augusto \nÓtima...,5,pt,Floresta das escadinhas,bem recebidas augusto ótima comida ótimo preço...
...,...,...,...,...,...,...,...,...,...,...,...
13250,13250,Jerzy1969,0,They have lost attention to food quality and C...,undefined,undefined,We have stoopped for small meal with kids- had...,2,eng,Nicola,we have stoopped small meal with kids had tabl...
13251,13251,Valerie G,0,Try another restaurant,08/2011,none,We stayed in a hotel along the Avenida Liberda...,2,eng,Nicola,we stayed in hotel along the avenida liberdad ...
13252,13252,J and d.,0,Great location for Dining in Central Lisbon,09/2011,none,The location on a main square in Lisbon makes ...,4,eng,Nicola,the location on main square in lisbon makes th...
13253,13253,fridayinla,0,Average dining with below average service.,undefined,undefined,Although the restaurant has a charmingly class...,3,eng,Nicola,although the restaurant has charmingly classic...


In [8]:
N=1000
#df.groupby('Variables').head(N).reset_index(drop=True)
df = df.groupby('Review_Classification').sample(n=N)

In [9]:
df['star'] = 'Star'
df['rating'] = df['Review_Classification'].astype(str) + ' '+ df['star']

In [10]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (f'Device Availble: {DEVICE}')

Device Availble: cuda


In [11]:
le = LabelEncoder()
df['rating'] = le.fit_transform(df['rating'])
df.head()
print (le.classes_)

['1 Star' '2 Star' '3 Star' '4 Star' '5 Star']


In [12]:
(train_texts, train_labels,
 val_texts, val_labels,
 test_texts, test_labels) = train_valid_test_split(df, target = 'rating', train_size=0.75, valid_size=0.1, test_size=0.15)

In [13]:
train_texts = train_texts['comentario_limpo'].to_list()
train_labels = train_labels.to_list()
val_texts = val_texts['comentario_limpo'].to_list()
val_labels = val_labels.to_list()
test_texts = test_texts['comentario_limpo'].to_list()
test_labels = test_labels.to_list()

In [14]:
class DataLoader(torch.utils.data.Dataset):
    def __init__(self, sentences=None, labels=None):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

        if bool(sentences):
            self.encodings = self.tokenizer(self.sentences,
                                            truncation = True,
                                            padding = True)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        if self.labels == None:
            item['labels'] = None
        else:
            item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.sentences)

    def encode(self, x):
        return self.tokenizer(x, return_tensors = 'pt').to(DEVICE)

In [15]:
train_dataset = DataLoader(train_texts, train_labels)
val_dataset = DataLoader(val_texts, val_labels)
test_dataset = DataLoader(test_texts, test_labels)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [16]:
print (train_dataset.__getitem__(0))

{'input_ids': tensor([   101,  86070,  46478,  11566,  10213,  91878,  16114,  59888,  12292,
         21323,  10147,  28642,  12081,  13621,  13326,  10570,  10758,  92394,
         10451,  25346,  24860,  54136,  10107,  39327,  11669,  35228,  28998,
         10129, 104767,  39368,  53953,  44967,  10229,  21000,  23585,  24301,
         78592,  61047,  20076,  13386,  30776,  10107,  11633,  10139,  79308,
         24832,  63915,  53953,  14541,  24832,  63915,  21323,  21427,  37984,
         19886,  22866,  24301,  44967,  30439,  10266,  30619,  25954,  53953,
         10178,  32657,  21052,  12656,  26978,  19341,  24832,  10310,  23821,
         21427,  21000,  16627,  26418,  53319,  10448,  14386,  47827,  19132,
         16942,  30743,  14865,  40754,  30632,  75921,  83642,  17650,  17339,
         58495,  11181,  40754,  30632,  20210,  10354,  35562,  12476,  10483,
         30518,  53524,  11726,  33972,    175,  17927,  58891,  55325,  75921,
         14541,  11153,  1

In [17]:
f1 = evaluate.load('f1')
accuracy = evaluate.load('accuracy')
precision = evaluate.load('precision')
recall = evaluate.load('recall')
def compute_metrica(eval_pred):
    metrics_dict = {}
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    metrics_dict.update(f1.compute(predictions = predictions, references = labels, average = 'macro'))
    metrics_dict.update(accuracy.compute(predictions = predictions, references = labels))
    metrics_dict.update(precision.compute(predictions = predictions, references = labels, average = 'macro'))
    metrics_dict.update(recall.compute(predictions = predictions, references = labels, average = 'macro'))
    return metrics_dict

In [18]:
id2label = {idx:label for idx, label in enumerate(le.classes_)}
label2id = {label:idx for idx, label in enumerate(le.classes_)}

In [19]:
config = AutoConfig.from_pretrained('bert-base-multilingual-cased',
                                    num_labels = 5,
                                    id2label = id2label,
                                    label2id = label2id)
model = AutoModelForSequenceClassification.from_config(config)
print (config)
print (model)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "1 Star",
    "1": "2 Star",
    "2": "3 Star",
    "3": "4 Star",
    "4": "5 Star"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "1 Star": 0,
    "2 Star": 1,
    "3 Star": 2,
    "4 Star": 3,
    "5 Star": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.52.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

BertForSequence

In [20]:
training_args = TrainingArguments(
    output_dir='/ver/out',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.05,
    report_to='none',
    logging_dir='/ver/log',
    logging_steps=50)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrica)

In [22]:
trainer.train()

Step,Training Loss
50,1.648800
100,1.617600
150,1.589400
200,1.547100
250,1.433400
300,1.407200
350,1.296500
400,1.280800
450,1.409900
500,1.175100


TrainOutput(global_step=1180, training_loss=1.0656689950975322, metrics={'train_runtime': 3407.3451, 'train_samples_per_second': 11.006, 'train_steps_per_second': 0.346, 'total_flos': 9866930342400000.0, 'train_loss': 1.0656689950975322, 'epoch': 10.0})

In [23]:
eval_results = trainer.predict(test_dataset)

In [24]:
print (eval_results.predictions)

[[ 3.4801164   2.7542393  -0.82184845 -3.0159602  -2.7270517 ]
 [ 4.4073153   1.5831296  -1.5324725  -2.868067   -1.5839921 ]
 [-2.445786   -2.7738237  -1.5539578   2.8553004   3.9601846 ]
 ...
 [ 1.2142982   3.5393097   0.99203134 -2.577594   -3.6059074 ]
 [-3.2169557  -2.3616734   0.42659736  3.7110364   1.4635751 ]
 [-0.5338993   2.7901669   2.6361036  -1.6483877  -3.6137974 ]]


In [25]:
print (eval_results.label_ids)

[1 0 4 4 4 4 4 2 1 1 2 1 4 4 0 2 1 1 0 4 4 2 1 1 1 0 2 4 2 0 1 3 0 3 0 4 3
 4 2 4 4 3 1 0 4 3 4 4 3 2 3 1 4 1 4 0 2 1 1 2 1 0 2 3 2 4 3 0 4 1 3 3 2 4
 2 3 3 2 4 1 4 3 2 2 4 2 3 4 1 3 0 3 0 2 1 3 4 3 0 4 4 2 4 4 0 4 2 3 3 4 1
 0 4 0 1 1 0 2 1 0 3 2 2 2 2 3 2 0 2 2 2 4 2 2 1 4 3 2 2 1 4 4 1 2 0 2 0 4
 1 0 1 1 3 1 4 3 0 1 1 3 3 3 0 0 2 4 3 3 3 3 3 3 1 4 1 4 1 1 4 0 1 1 2 4 3
 0 3 2 0 0 3 1 0 0 1 1 3 0 1 3 0 1 1 4 0 1 2 1 1 2 4 1 1 3 3 3 2 1 0 3 4 2
 3 3 1 2 3 3 2 4 1 1 4 1 2 3 0 3 0 1 3 0 2 3 2 3 1 0 1 3 3 3 1 4 1 4 3 3 4
 0 1 4 0 1 1 0 3 4 4 0 1 1 1 4 4 2 0 0 3 3 0 1 3 0 2 4 0 1 3 0 1 1 1 1 3 4
 1 1 1 1 2 1 2 2 4 3 4 1 0 3 3 1 2 2 4 1 2 4 4 0 1 1 4 4 3 1 3 0 2 4 3 1 3
 2 0 4 0 0 1 4 4 0 1 2 0 3 4 0 3 2 2 2 2 4 4 0 0 3 4 1 1 0 1 0 2 4 4 4 1 2
 3 0 1 3 0 4 2 2 3 2 3 2 3 0 3 2 4 3 4 2 2 4 0 2 3 3 1 1 3 0 0 4 2 1 2 0 0
 0 4 0 4 1 0 4 0 3 0 4 4 0 1 4 1 1 2 4 2 2 2 1 1 2 4 2 2 1 3 0 2 1 3 0 1 2
 0 4 1 4 3 1 1 1 0 3 1 3 4 0 1 2 3 4 3 1 0 3 2 0 3 0 2 4 4 3 0 4 2 3 0 1 4
 1 3 2 0 1 4 1 2 2 3 2 0 

In [26]:
print (eval_results.metrics)

{'test_loss': 1.2804114818572998, 'test_f1': 0.54643394229387, 'test_accuracy': 0.5413333333333333, 'test_precision': 0.5593102953845067, 'test_recall': 0.541844151851321, 'test_runtime': 21.8323, 'test_samples_per_second': 34.353, 'test_steps_per_second': 1.099}


In [27]:
label2id_mapper = model.config.id2label
proba = softmax(torch.from_numpy(eval_results.predictions))
pred = [label2id_mapper[i] for i in torch.argmax(proba, dim = -1).numpy()]
actual = [label2id_mapper[i] for i in eval_results.label_ids]

In [28]:
class_report = classification_report(actual, pred, output_dict = True)
pd.DataFrame(class_report)

,1 Star,2 Star,3 Star,4 Star,5 Star,accuracy,macro avg,weighted avg
precision,0.672269,0.490798,0.458101,0.451807,0.723577,0.541333,0.559310,0.559088
recall,0.536913,0.503145,0.561644,0.510204,0.597315,0.541333,0.541844,0.541333
f1-score,0.597015,0.496894,0.504615,0.479233,0.654412,0.541333,0.546434,0.546120
support,149.000000,159.000000,146.000000,147.000000,149.000000,0.541333,750.000000,750.000000
